In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline

Let's get the database credentials we need.

In [ ]:
import os

OSM_USER=os.getenv('POSTGRES_USER')
OSM_PASS=os.getenv('POSTGRES_PASS')

# This is a hack to be able to run osm2pgsql without interactively entering the password
os.environ['PGPASSWORD'] = OSM_PASS

--hstore
adds any tags not already in a conventional column to a hstore column. With the standard stylesheet this would result in tags like highway appearing in a conventional column while tags not in the style like name:en or lanes:forward would appear only in the hstore column.

--hstore-add-index
adds an index to the hstore columns

In [ ]:
%%bash 

osm2pgsql --hstore --latlong \
    --database osm \
    --username $POSTGRES_USER \
    --host osm_db \
    /data/saint_kitts_and_nevis-latest.osm.pbf

In [ ]:
import pandas as pd
import geopandas as gpd
import sqlalchemy

Learned all of this from https://songhuiming.github.io/pages/2017/04/02/jupyter-and-pandas-display/

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)
# Display all text in a text without truncation
pd.set_option('display.max_colwidth', -1)

In [ ]:
connection_string = 'postgresql://{}:{}@osm_db:5432/osm'.format(OSM_USER, OSM_PASS)

In [ ]:
connection_string

In [ ]:
engine = sqlalchemy.create_engine(connection_string)

# What are the OSM Tables?
- planet_osm_point: which contains points of interest such as restaurants, hospitals, schools, supermarkets and addresses
- planet_osm_lines: contains roads and streets
- planet_osm_polygons: contains lakes, building footprints, administrative boundaries such as towns and cities

**Source: https://www.bostongis.com/PrinterFriendly.aspx?content_name=loading_osm_postgis**

# How do I extract values from the tags column?
SELECT osm_id, tags ->'website' AS website FROM planet_osm_polygon where tags -> 'website' is not null limit 20;

In [ ]:
query = """
SELECT osm_id, "addr:housename", "addr:housenumber", amenity, building, 
    leisure, name, office, religion, 
    service, shop, sport, surface, tourism, 
    way, tags ->'website' AS website, tags
        FROM planet_osm_polygon 
            WHERE name IS NOT NULL 
                OR tags ->'website' IS NOT NULL
                OR amenity IS NOT NULL
                OR building IS NOT NULL
                OR leisure IS NOT NULL
                OR office IS NOT NULL
                OR religion IS NOT NULL
                OR service IS NOT NULL
                OR shop IS NOT NULL
                OR sport IS NOT NULL
                OR tourism IS NOT NULL
                ORDER BY RANDOM()
"""
osm_polygon = gpd.read_postgis(sql=query,
                con=engine, geom_col='way', index_col='osm_id')

In [ ]:
osm_polygon.tail()

In [ ]:
import qgrid

In [ ]:
qgrid.show_grid(osm_polygon.drop(columns=['way']))

In [ ]:
query = """
SELECT osm_id, "addr:housename", "addr:housenumber", amenity, building, 
    leisure, name, office, religion, 
    service, shop, sport, surface, tourism, 
    way, tags ->'website' AS website, tags
        FROM planet_osm_point
            WHERE name IS NOT NULL 
                OR tags ->'website' IS NOT NULL
                OR amenity IS NOT NULL
                OR building IS NOT NULL
                OR leisure IS NOT NULL
                OR office IS NOT NULL
                OR religion IS NOT NULL
                OR service IS NOT NULL
                OR shop IS NOT NULL
                OR sport IS NOT NULL
                OR tourism IS NOT NULL
                ORDER BY RANDOM()
"""
osm_point = gpd.read_postgis(sql=query,
                con=engine, geom_col='way',
                index_col='osm_id')

In [ ]:
qgrid.show_grid(osm_point.drop(columns=['way']))

In [ ]:
query = """
SELECT osm_id, "addr:housename", "addr:housenumber", amenity, building, 
    leisure, name, office, religion, 
    service, shop, sport, surface, tourism, 
    way, tags ->'website' AS website, tags
        FROM planet_osm_line
            WHERE name IS NOT NULL 
                OR tags ->'website' IS NOT NULL
                OR amenity IS NOT NULL
                OR building IS NOT NULL
                OR leisure IS NOT NULL
                OR office IS NOT NULL
                OR religion IS NOT NULL
                OR service IS NOT NULL
                OR shop IS NOT NULL
                OR sport IS NOT NULL
                OR tourism IS NOT NULL
                ORDER BY RANDOM()
"""
osm_line = gpd.read_postgis(sql=query,
                con=engine, geom_col='way',
                index_col='osm_id')

In [ ]:
qgrid.show_grid(osm_line.drop(columns=['way']))